# Sales file

In [85]:
# import required packages 
# Missing value treatment
# Null value treatment
# Null imputation

In [ ]:
!pip install squarify

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from datetime import datetime
import squarify
from sqlalchemy import create_engine

In [ ]:
data = pd.read_csv("Sales.csv")

In [ ]:
data

In [ ]:
data.isnull().sum()

In [ ]:
data.isnull().mean() * 100 # percentage of empty

In [ ]:
# fill missing values
data['Delivery Date'].fillna(0, inplace=True) # More than 60% we can't able to treat the data. so we are replacing blank insted of dropping we can use and fill 0

In [ ]:
data.duplicated()

In [ ]:
# removing duplicates of order number because of we need unique value
data.drop_duplicates('Order Number', inplace = True)

In [ ]:
data['Order Date'] = pd.to_datetime(data['Order Date'], errors = 'coerce')
data['Delivery Date'] = pd.to_datetime(data['Delivery Date'], errors = 'coerce')

In [ ]:
#data['Order Date'].replace(['0', 'NaT'], None)

In [ ]:
# Convert 'Order Date' and 'Delivery Date' columns to datetime format
data['Order Date'] = pd.to_datetime(data['Order Date'], errors='coerce')
data['Delivery Date'] = pd.to_datetime(data['Delivery Date'], errors='coerce')

# Check for missing values in each column
missing_values = data.isnull().sum()

# Display missing values
missing_values


In [ ]:
data

# Customer file

In [ ]:
#cus_data = pd.read_csv('Customers.csv', encoding='utf-8')
cus_data = pd.read_csv('Customers.csv', encoding='latin-1')

In [ ]:
cus_data

In [ ]:
cus_data['Birthday'] = pd.to_datetime(cus_data['Birthday'], format="%M/%d/%Y", errors='coerce').dt.date
cus_data

In [ ]:
cus_data.isnull().sum()

In [ ]:
# Drop mising values
cus_data.isnull().mean() * 100

In [ ]:
# this missing value wn't be big impact , but still we wil remove this
cus_data.dropna(inplace=True)

In [ ]:
cus_data.isnull().sum()

In [ ]:
# drop unwanted columns
cus_data.drop(['State Code', 'Zip Code'], axis=1, inplace=True)

In [ ]:
cus_data

In [ ]:
sns.set_style("dark")
plt.style.use("dark_background")
plt.figure(figsize=(4,4))
gender_counts = cus_data['Gender'].value_counts()
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=100, colors=["blue", "red"], wedgeprops={'width':0.5})
plt.title("Count of Gender")
plt.show()

plt.subplot(2, 2, 2)

sns.countplot(x=cus_data["Gender"], data=cus_data, hue="Gender")
plt.xlabel("Gender")
plt.ylabel("Counts")
plt.title("Counts as per Gender")

plt.show()

In [ ]:
cus_data

In [ ]:
# Set the dark background and grid style
sns.set_style("dark")
plt.style.use("dark_background")

unique_countries = cus_data["Country"].nunique()
custom_colors = ['red', 'blue', 'yellow', 'white', 'grey', 'violet', 'orange', 'green']#[:unique_countries]

# Set up the figure size
plt.figure(figsize=(12, 5))
sns.countplot(x="Country", hue="Country", data=cus_data, palette=custom_colors, dodge=False)
plt.xlabel("Country")
plt.ylabel("Counts")
plt.title("Counts per Country")
plt.legend([], [], frameon=False)
plt.show()


# Exchange Rate file

In [ ]:
ex_data = pd.read_csv("Exchange_Rates.csv")

In [ ]:
ex_data

In [ ]:
ex_data.rename(columns={"Currency":"Currency Code"},inplace=True)
ex_data['Date'] = pd.to_datetime(ex_data['Date'], format="%m/%d/%Y", errors='coerce')

In [ ]:
ex_data

In [ ]:
ex_data.isnull().sum()

In [ ]:
sns.set_style("dark")
plt.style.use("dark_background")
plt.figure(figsize=(13, 6))
sns.countplot(data=ex_data, x='Currency Code')

plt.xlabel("Currency Code")
plt.ylabel("Count")
plt.title("Count as per currency")

plt.show()


# Stores file

In [ ]:
st_data = pd.read_csv("Stores.csv")

In [ ]:
st_data

In [ ]:
st_data.isnull().sum()

In [ ]:
st_data.fillna(0)

In [ ]:
st_data['Open Date'] = pd.to_datetime(st_data['Open Date'], format="%m/%d/%Y", errors='coerce').dt.date

In [ ]:
st_data

In [ ]:
st_data = st_data.replace({np.nan: None})
print(st_data)

# Product file

In [ ]:
pro_data = pd.read_csv("Products.csv")

In [ ]:
pro_data

In [ ]:
pro_data.isnull().sum()

In [ ]:
pro_data

In [ ]:
pro_data.drop(['SubcategoryKey'], axis =1, inplace=True)

In [ ]:
pro_data["Unit Cost USD"] = pro_data["Unit Cost USD"].str.replace("$", "").str.replace(",", "")
pro_data["Unit Price USD"] = pro_data["Unit Price USD"].str.replace("$", "").str.replace(",", "")
pro_data["Unit Price USD"] = pro_data["Unit Price USD"].astype(float)
pro_data["Unit Cost USD"] = pro_data["Unit Cost USD"].astype(float)

In [ ]:

brand_category = pro_data.groupby(['Brand', 'Category']).size().unstack().fillna(0)
brand_category.plot(kind='bar', stacked=True, figsize=(10, 7), colormap="Set1")
plt.title("Count of Brands and Categories")
plt.xlabel("Brand")
plt.ylabel("Counts")
plt.legend(title="Category")
plt.show()


# My SQL Connector

In [ ]:

conn=mysql.connector.connect(host="localhost", user="root", password="root123",database="sales_cust")
my_cursor = conn.cursor()
my_cursor.execute("CREATE DATABASE IF NOT EXISTS sales_cust")
print("database created successfully")

In [ ]:
# In customer column birthday column is not correct so droppe exitsing column and inserted new column
with conn.cursor() as cursor:
    
    cursor.execute("DROP TABLE IF EXISTS customer_details")

conn.commit()
conn.close()

In [ ]:
#sales table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS sales_details(
                  Order_Number INT,
                  Line_Item INT,
                  Order_Date VARCHAR(255),
                  Delivery_Date VARCHAR(255),
                  CustomerKey INT,
                  StoreKey INT,
                  ProductKey INT,
                  Quantity INT,
                  Currency_Code VARCHAR(255)
                  )''')
print("Table Created successfully")

In [ ]:
#insert values into sales table
insert_query ='''INSERT INTO sales_details(
                  Order_Number,
                  Line_Item,
                  Order_Date,
                  Delivery_Date,
                  CustomerKey,
                  StoreKey,
                  ProductKey,
                  Quantity,
                  Currency_Code)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s)'''
data=data.values.tolist()
my_cursor.executemany(insert_query,data)
print("Inserted successfully")

In [227]:
#customer table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS customer_details(
                  CustomerKey INT,
                  Gender VARCHAR(255),
                  Name VARCHAR(255),
                  City VARCHAR(255),
                  State VARCHAR(255),
                  Country VARCHAR(255),
                  Continent VARCHAR(300),
                  Birthday VARCHAR(255)
                  )''')
print("Table Created successfully")

Table Created successfully


In [229]:
insert_query = '''INSERT INTO customer_details(
                  CustomerKey,
                  Gender,
                  Name,
                  City,
                  State,
                  Country,
                  Continent,
                  Birthday
                  )
                  VALUES(%s,%s,%s,%s,%s,%s,%s,%s)'''
data = cus_data.values.tolist()
my_cursor.executemany(insert_query, data)
print("Inserted successfully")


Inserted successfully


In [ ]:
# exchange table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS exchange_details(
                  Date VARCHAR(255),
                  Currency_Code VARCHAR(255),
                  Exchange FLOAT
                  )''')
print("Table Created successfully")

In [ ]:
# insert values into exchange table
insert_query ='''INSERT INTO exchange_details(
                  Date,
                 Currency_Code,
                  Exchange)
                  VALUES (%s, %s, %s)'''
data=ex_data.values.tolist()
my_cursor.executemany(insert_query,data)
print("Inserted successfully")

In [ ]:
#store table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS stores_details(
                  StoreKey INT,
                  Country VARCHAR(255),
                  State VARCHAR(255),
                  Square_Meters FLOAT,
                  Open_Date VARCHAR(255)
                  )''')
print("Table Created successfully")

In [192]:
##insert values into Store table
insert_query ='''INSERT INTO stores_details(
                 StoreKey,
                  Country,
                  State,
                  Square_Meters,
                  Open_Date)
                  VALUES (%s, %s, %s, %s, %s)'''
data=st_data.values.tolist()
my_cursor.executemany(insert_query,data)
print("Inserted successfully")

Inserted successfully


In [194]:
# create Products table 
my_cursor.execute('''CREATE TABLE IF NOT EXISTS product_details(
                  ProductKey INT,
                  Product_Name VARCHAR(255),
                  Brand VARCHAR(255),
                  Color VARCHAR(255),
                  Unit_Cost_USD FLOAT,
                  Unit_Price_USD FLOAT,
                  Subcategory VARCHAR(255),
                  CategoryKey VARCHAR(255),
                  Category VARCHAR(255)
                  )''')
print("Table Created successfully")

Table Created successfully


In [196]:
#insert values into Products table
insert_query ='''INSERT INTO product_details(
                  ProductKey,
                  Product_Name,
                  Brand,
                  Color,
                  Unit_Cost_USD,
                  Unit_Price_USD,
                  Subcategory,
                  CategoryKey,
                  Category)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s)'''
data=pro_data.values.tolist()
my_cursor.executemany(insert_query,data)
print("Inserted successfully")

Inserted successfully


# 10 Query

In [199]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)


def dataframe_conn (query, connection):
    cursor = connection.cursor()
    cursor.execute(query)
    columns = [col[0] for col in cursor.description] 
    data = cursor.fetchall()  
    cursor.close()
    return pd.DataFrame(data, columns=columns)  


customer_details = dataframe_conn("SELECT * FROM customer_details", conn)
exchange_details = dataframe_conn("SELECT * FROM exchange_details", conn)
product_details = dataframe_conn("SELECT * FROM product_details", conn)
sales_details = dataframe_conn("SELECT * FROM sales_details", conn)
stores_details = dataframe_conn("SELECT * FROM stores_details", conn)

# Close the connection
conn.close()


In [200]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)

# Initialize cursor
cursor = conn.cursor()

try:
    # 1. Overall female count
    cursor.execute("SELECT COUNT(Gender) AS Female_count FROM customer_details WHERE Gender = 'Female'")
    female_count = cursor.fetchone()[0]
    print("Female count:", female_count)
    
       
finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Female count: 7514


In [201]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
    # 2. Overall male count
    cursor.execute("SELECT COUNT(Gender) AS Male_count FROM customer_details WHERE Gender = 'Male'")
    male_count = cursor.fetchone()[0]
    print("Male count:", male_count)
    
finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()
    
   

Male count: 7742


In [202]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
    # 3. Count of customers in each country
    cursor.execute("""
        SELECT sd.Country, COUNT(DISTINCT c.CustomerKey) AS customer_count 
        FROM sales_details c 
        JOIN stores_details sd ON c.StoreKey = sd.StoreKey
        GROUP BY sd.Country 
        ORDER BY customer_count DESC
    """)
    country_customer_counts = cursor.fetchall()
    print("Customer count by country:")
    for row in country_customer_counts:
        print(row)
    
finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

Customer count by country:
('United States', 5179)
('Online', 4547)
('United Kingdom', 1424)
('Canada', 1042)
('Germany', 1008)
('Australia', 657)
('Netherlands', 483)
('Italy', 475)
('France', 389)


In [203]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
    # 4. Overall count of customers
    cursor.execute("SELECT COUNT(DISTINCT CustomerKey) AS customer_count FROM sales_details")
    total_customers = cursor.fetchone()[0]
    print("Total customers:", total_customers)

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Total customers: 11887


In [204]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
    # 5. Count of stores in each country
    cursor.execute("""
        SELECT Country, COUNT(StoreKey) 
        FROM stores_details
        GROUP BY Country 
        ORDER BY COUNT(StoreKey) DESC
    """)
    stores_by_country = cursor.fetchall()
    print("Stores by country:")
    for row in stores_by_country:
        print(row)
    


finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Stores by country:
('United States', 72)
('Germany', 27)
('France', 21)
('United Kingdom', 21)
('Australia', 18)
('Canada', 15)
('Netherlands', 15)
('Italy', 9)
('Online', 3)


In [208]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
    # 6. Store-wise sales
    cursor.execute("""
        SELECT s.StoreKey, sd.Country, SUM(Unit_Price_USD * s.Quantity) AS total_sales_amount 
        FROM product_details pd
        JOIN sales_details s ON pd.ProductKey = s.ProductKey
        JOIN stores_details sd ON s.StoreKey = sd.StoreKey
        GROUP BY s.StoreKey, sd.Country
    """)
    store_sales = cursor.fetchall()
    print("Store-wise sales:")
    
    


finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Store-wise sales:


In [209]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
    # 7. Overall selling amount
    cursor.execute("""
        SELECT SUM(Unit_Price_USD * sd.Quantity) AS total_sales_amount 
        FROM product_details pd
        JOIN sales_details sd ON pd.ProductKey = sd.ProductKey
    """)
    total_sales_amount = cursor.fetchone()[0]
    print("Total selling amount:", total_sales_amount)

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Total selling amount: 189651225.74551153


In [210]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
    # 8. Brand count
    cursor.execute("SELECT Brand, COUNT(Brand) AS brand_count FROM product_details GROUP BY Brand")
    brand_counts = cursor.fetchall()
    print("Brand counts:")
    for row in brand_counts:
        print(row)


finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Brand counts:
('Contoso', 1420)
('Wide World Importers', 346)
('Northwind Traders', 94)
('Adventure Works', 384)
('Southridge Video', 384)
('Litware', 528)
('Fabrikam', 534)
('Proseware', 488)
('A. Datum', 264)
('The Phone Company', 304)
('Tailspin Toys', 288)


In [212]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
     # 9. Cost price, selling price difference, and profit percentage
    cursor.execute("""
        SELECT Unit_price_USD, Unit_Cost_USD, 
               ROUND((Unit_price_USD - Unit_Cost_USD), 2) AS diff,
               ROUND(((Unit_price_USD - Unit_Cost_USD) / Unit_Cost_USD) * 100, 2) AS profit_percent
        FROM product_details
    """)
    profit_data = cursor.fetchall()
    print("Profit data (cost price, selling price difference, and profit percent):")
    for row in profit_data:
        print(row)


finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Profit data (cost price, selling price difference, and profit percent):
(12.99, 6.62, 6.37, 96.22)
(12.99, 6.62, 6.37, 96.22)
(14.52, 7.4, 7.12, 96.22)
(21.57, 11.0, 10.57, 96.09)
(21.57, 11.0, 10.57, 96.09)
(21.57, 11.0, 10.57, 96.09)
(21.57, 11.0, 10.57, 96.09)
(59.99, 30.58, 29.41, 96.17)
(59.99, 30.58, 29.41, 96.17)
(59.99, 30.58, 29.41, 96.17)
(59.99, 30.58, 29.41, 96.17)
(77.68, 35.72, 41.96, 117.47)
(77.68, 35.72, 41.96, 117.47)
(77.68, 35.72, 41.96, 117.47)
(77.68, 35.72, 41.96, 117.47)
(109.95, 50.56, 59.39, 117.46)
(109.95, 50.56, 59.39, 117.46)
(109.95, 50.56, 59.39, 117.46)
(109.95, 50.56, 59.39, 117.46)
(134.0, 61.62, 72.38, 117.46)
(134.0, 61.62, 72.38, 117.46)
(134.0, 61.62, 72.38, 117.46)
(134.0, 61.62, 72.38, 117.46)
(199.9, 91.93, 107.97, 117.45)
(199.9, 91.93, 107.97, 117.45)
(199.9, 91.93, 107.97, 117.45)
(199.9, 91.93, 107.97, 117.45)
(199.9, 91.93, 107.97, 117.45)
(255.0, 84.49, 170.51, 201.81)
(255.0, 84.49, 170.51, 201.81)
(255.0, 84.49, 170.51, 201.81)
(255.0, 

In [215]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)
cursor = conn.cursor()
try:
    # 10. Brand-wise selling amount
    cursor.execute("""
        SELECT Brand, ROUND(SUM(Unit_price_USD * sd.Quantity), 2) AS sales_amount
        FROM product_details pd 
        JOIN sales_details sd ON pd.ProductKey = sd.ProductKey 
        GROUP BY Brand
    """)
    brand_sales = cursor.fetchall()
    print("Brand-wise sales amounts:")
    for row in brand_sales:
        print(row)


finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


Brand-wise sales amounts:
('Contoso', 36651484.93)
('A. Datum', 5027707.99)
('Adventure Works', 39474135.81)
('Fabrikam', 23385178.08)
('Wide World Importers', 31805514.25)
('Tailspin Toys', 2313544.46)
('Southridge Video', 8646383.57)
('Litware', 9139092.64)
('Proseware', 10777068.32)
('The Phone Company', 18228128.0)
('Northwind Traders', 4202987.69)


# Querying and Merging Multiple Tables Convert as CSV file

In [217]:
conn = mysql.connector.connect(
    host='localhost',       
    user="root",    
    password='root123',
    database='sales_cust'
)


def dataframe_conn (query, connection):
    cursor = connection.cursor()
    cursor.execute(query)
    columns = [col[0] for col in cursor.description] 
    data = cursor.fetchall()  
    cursor.close()
    return pd.DataFrame(data, columns=columns)  


customer_details = dataframe_conn("SELECT * FROM customer_details", conn)
exchange_details = dataframe_conn("SELECT * FROM exchange_details", conn)
product_details = dataframe_conn("SELECT * FROM product_details", conn)
sales_details = dataframe_conn("SELECT * FROM sales_details", conn)
stores_details = dataframe_conn("SELECT * FROM stores_details", conn)

# Close the connection
conn.close()
